In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import oauth2client

In [2]:
def load_data(sheet_name):
    # サービスアカウントキーのJSONファイルへのパス
    json_keyfile = 'ryuon-equipment-38a59fa0f789.json'

    # 認証情報の設定
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
    gc = gspread.authorize(credentials)

    # スプレッドシートを開く
    spreadsheet_key = '1LZiE6wf8VTwFoVxeEBTx3OXOKx6vQA5IaItE7NQYZTs'
    worksheet_ksr = gc.open_by_key(spreadsheet_key).worksheet('ksr'+sheet_name)
    worksheet_ssr = gc.open_by_key(spreadsheet_key).worksheet('ssr'+sheet_name)
    worksheet_category = gc.open_by_key(spreadsheet_key).worksheet('ability-category')

    # シートデータの読み込み
    data_ksr = worksheet_ksr.get_all_records()
    data_ssr = worksheet_ssr.get_all_records()
    data_category = worksheet_category.get_all_records()

    # 辞書のリストからDataFrameに変換
    df_ksr = pd.DataFrame(data_ksr)
    df_ssr = pd.DataFrame(data_ssr)
    df_category = pd.DataFrame(data_category)

    # シートデータを結合して1つのDataFrameにする
    df = pd.concat([df_ksr, df_ssr])

    return df,df_category


In [3]:
df = load_data('武器')[0]
df_reloaded = df.copy()

df_category_ori = load_data('武器')[1]
df_category = df_category_ori.copy()

output = df_reloaded.copy()
df_status = df_reloaded.copy()

#本番output,df_statusで置き換えることで,Noneが出力しないようにしてる
df_status.replace("" ,np.nan ,inplace=True)

In [4]:
output

,装備名,装備番号,レアリティ,体力,攻撃力,防御力,会心率,回避率,命中率,アビリティ,アビリティカテゴリ
0,龍の鉄拳・極,K001,KSR,,323,,11.7,,,,
1,釘原のナイフ,K002,KSR,,610,,,,10.8,ドンパチ・タイマン時に東城会の特性を持つ敵が2人以上のときスキルCTを8%加速,BSCT加速
2,真島の解体用ペンチ,K003,KSR,,615,,5.9,,,体属性の味方人数×5%攻撃力上昇,攻撃力上昇
3,アシリパの弓,K004,KSR,,511,,,,11.4,通常攻撃時18%の確率で3.0秒間の麻痺付与,状態異常付与
4,尾形の銃,K005,KSR,,533,,9.3,,,街の住人の特性を持つ敵人数×5.0%会心率を上昇(ドンパチ・タイマン時),会心率上昇
...,...,...,...,...,...,...,...,...,...,...,...
95,カランビットナイフ,S096,SSR,,398,,,8.7,,HP50％以上のときスキルCT8％加速,BSCT加速
96,ラウ・カーロンの薙刀,S097,SSR,,423,,,,9.9,防御タイプの敵人数×5%攻撃力上昇,攻撃力上昇
97,佐川の金属バット,S098,SSR,3050,,,,6,,状態異常になる確率を10％減少,被状態異常確率低減
98,小清水の竹刀,S099,SSR,,443,,10.8,,,バトルスキルのヒートゲージ上昇量20%上昇,ヒートゲージ上昇


In [8]:
rare_filter = None
rare_list = []

rare_list.append('SSR')

rare_filter = output['レアリティ'].isin(rare_list)
output = output[rare_filter]

output

,装備名,装備番号,レアリティ,体力,攻撃力,防御力,会心率,回避率,命中率,アビリティ,アビリティカテゴリ
27,サイリウム,S028,SSR,1296,,305,,,,HP50%以上のときスキルクールタイムが8%上昇,BSCT加速
38,谷村のリボルバー,S039,SSR,,,378,10.7,,,通常攻撃時12%の確率で6.0秒間の封印付与,状態異常付与
81,城戸の拳銃,S082,SSR,945,,320,,,,通常攻撃時12％の確率で敵の攻撃力を9秒間-12％減少,敵攻撃ダウン
83,馬場の狙撃銃,S084,SSR,,,330,,,5.9,攻撃タイプの敵人数×5%防御力上昇,防御力上昇
87,天童のメリケンサック,S088,SSR,,,338,10.7,,,補助タイプの敵人数×5％防御力上昇,防御力上昇
90,GBホームズのグローブ,S091,SSR,,272,335,,,,打撲状態の敵に対しての攻撃力が25％上昇,攻撃力上昇


In [7]:
status = True
status = status & (df_status['防御力'] > 0)
output = output[status]

output

/var/folders/th/68_z619x1yx_ddy_dcxgry300000gn/T/ipykernel_12921/1618096406.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  output = output[status]


IndexError: indices are out-of-bounds

In [15]:
import streamlit as st
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import oauth2client

def load_data(sheet_name):
    # サービスアカウントキーのJSONファイルへのパス
    json_keyfile = 'ryuon-equipment-38a59fa0f789.json'

    # 認証情報の設定
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
    gc = gspread.authorize(credentials)

    # スプレッドシートを開く
    spreadsheet_key = '1LZiE6wf8VTwFoVxeEBTx3OXOKx6vQA5IaItE7NQYZTs'
    worksheet_ksr = gc.open_by_key(spreadsheet_key).worksheet('ksr'+sheet_name)
    worksheet_ssr = gc.open_by_key(spreadsheet_key).worksheet('ssr'+sheet_name)
    worksheet_category = gc.open_by_key(spreadsheet_key).worksheet('ability-category')

    # シートデータの読み込み
    data_ksr = worksheet_ksr.get_all_records()
    data_ssr = worksheet_ssr.get_all_records()
    data_category = worksheet_category.get_all_records()

    # 辞書のリストからDataFrameに変換
    df_ksr = pd.DataFrame(data_ksr)
    df_ssr = pd.DataFrame(data_ssr)
    df_category = pd.DataFrame(data_category)

    # シートデータを結合して1つのDataFrameにする
    df = pd.concat([df_ksr, df_ssr])

    return df,df_category

df = load_data('武器')[0]
df_reloaded = df.copy()

df_category_ori = load_data('武器')[1]
df_category = df_category_ori.copy()

output = df_reloaded.copy()
df_status = df_reloaded.copy()

#本番output,df_statusで置き換えることで,Noneが出力しないようにしてる
df_status.replace("" ,np.nan ,inplace=True)


# #レアリティフィルタ
# rare_filter = None
# rare_list = []

# rare_list.append('SSR')
# rare_filter = output['レアリティ'].isin(rare_list)
# output = output[rare_filter]

#ステータスフィルタ

status = True
status = status & (df_status['攻撃力'] > 0)
output = output[status]

# print("output の行数:", len(output))
# print("status の行数:", len(status))

# print(status)

# output[status]
# output

,装備名,装備番号,レアリティ,体力,攻撃力,防御力,会心率,回避率,命中率,アビリティ,アビリティカテゴリ
0,龍の鉄拳・極,K001,KSR,,323,,11.7,,,,
1,釘原のナイフ,K002,KSR,,610,,,,10.8,ドンパチ・タイマン時に東城会の特性を持つ敵が2人以上のときスキルCTを8%加速,BSCT加速
2,真島の解体用ペンチ,K003,KSR,,615,,5.9,,,体属性の味方人数×5%攻撃力上昇,攻撃力上昇
3,アシリパの弓,K004,KSR,,511,,,,11.4,通常攻撃時18%の確率で3.0秒間の麻痺付与,状態異常付与
4,尾形の銃,K005,KSR,,533,,9.3,,,街の住人の特性を持つ敵人数×5.0%会心率を上昇(ドンパチ・タイマン時),会心率上昇
...,...,...,...,...,...,...,...,...,...,...,...
94,山田照のヌンチャク,S095,SSR,,369,,,,8.4,通常攻撃時10％の確率で4.0秒間の混乱付与,状態異常付与
95,カランビットナイフ,S096,SSR,,398,,,8.7,,HP50％以上のときスキルCT8％加速,BSCT加速
96,ラウ・カーロンの薙刀,S097,SSR,,423,,,,9.9,防御タイプの敵人数×5%攻撃力上昇,攻撃力上昇
98,小清水の竹刀,S099,SSR,,443,,10.8,,,バトルスキルのヒートゲージ上昇量20%上昇,ヒートゲージ上昇
